In [1]:
%pylab inline
import seaborn as sns
sns.set_context("talk")

import scipy.stats as st
import scipy.integrate
import ipywidgets as widgets

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


## Diskrete KL-Divergenz, binäres Zufallsexperiment

$$\begin{matrix}
P_1(x=0) = a & P_1(x=1) = 1-a \\
P_2(x=0) = b & P_2(x=1) = 1-b
\end{matrix}$$

$$\Rightarrow D_{\text{KL}}(P_1\|P_2) = a \log\frac ab + (1-a) \log \frac{1-a}{1-b}$$

In [7]:
@widgets.interact(a=(0,1,.01))
def toy_example(a=.3):
    b = linspace(1e-2,1-1e-2,100)
    
    figure(figsize=(10,5))
    plot(b, a*log(a/b) + (1-a)*log((1-a)/(1-b)))
    axvline(a, 0, .5, color='red')
    text(a, ylim()[1]/2+.1, "a", horizontalalignment='center')
    xlabel("b")
    ylabel("D({a,1-a}||{b,1-b})")
    xlim(0,1)
    ylim(0)
    sns.despine()
    show()

## Diskrete KL-Divergenz:

$$D_{\text{KL}}(P\|Q) = \!\!\!\!\!\!\sum_{m\,\in\,\text{supp}\{P(m)\}}\!\!\!\!\!\!P(m) \log \frac{P(m)}{Q(m)}$$

In [3]:
def KL_discrete(P, Q):
    support = P > 0
    return sum(P[support]*log(P[support]/Q[support]))

### KL-Divergenz zwischen Binomial- und Poission-Verteilung

* **Binomialverteilung**: Wahrscheinlichkeit, dass von $T$ gleichartigen Experimenten mit Erfolgswahrscheinlichkeit $p$ die Anzahl $m$ von Experimenten erfolgreich ist:

$$B\left(m\,|\,T,p\right) = \begin{pmatrix}T\\m\end{pmatrix} p^m (1-p)^{T-m}$$

* **Poission-Verteilung**: Wahrscheinlichkeit, in einem festen Zeitintervall $m$ Ereignisse zu beobachten, wenn $\lambda$ Ereignisse erwartet werden:

$$\textit{Poi}\left(m\,|\,\lambda\right) = \frac{\lambda^m}{m!}e^{-\lambda}$$

In [8]:
def dkl_example(T, p, lam):
    m = arange(T)
    P, Q = st.binom.pmf(m, T, p), st.poisson.pmf(m, lam)
    klPQ, klQP = KL_discrete(P, Q), KL_discrete(Q, P)
    
    figure(figsize=(12,6))
    bar(m-.2, P, .4, label="B(T={}, p={:.2f})".format(T, p))
    bar(m+.2, Q, .4, label="Poi(λ={})".format(lam))
    
    legend(loc="best")
    xlim(0-.5,m[-1])
    xlabel("m")
    ylabel("P(m)")
    title("D( B || Poi ) = {:.03f}   und   D( Poi || B ) = {:.03f}".format(klPQ, klQP))
    
    sns.despine()
    show()
    
interactive_plot = widgets.interactive(dkl_example,
    T = widgets.IntSlider(min=0, max=50, value=20),
    p = widgets.FloatSlider(min=0, max=1, step=.01, value=.2),
    lam = widgets.FloatSlider(min=0, max=20, step=.1, value=.6))

output = interactive_plot.children[-1]
output.layout.height = '410px'
interactive_plot

## Kontinuierliche KL-Divergenz:

$$D_{\text{KL}}(p\|q) = \!\!\!\!\int\limits_{\text{supp}\{p(m)\}}\!\!\!\! p(m) \log \frac{p(m)}{q(m)}\, \mathrm{d}m$$

In [5]:
def KL_continuous(p, q, mvals):
    return scipy.integrate.trapz(p*log(p/q), mvals)

### KL-Divergenz zwischen zwei Normalverteilungen

**Normalverteilung** mit Erwartungswert $\mu$ und Varianz $\sigma^2$:

$$N\left(m\,|\,\mu,\sigma\right) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left\{-\frac{(m - \mu)^2}{2\sigma^2}\right\}$$

In [6]:
def kkl_example(mu1, sigma1, mu2, sigma2):
    m = linspace(-10,10,150)
    p, q = st.norm.pdf(m,mu1,sigma1), st.norm.pdf(m,mu2,sigma2)
    klpq, klqp = KL_continuous(p, q, m), KL_continuous(q, p, m)
    
    figure(figsize=(12,6))
    plot(m, p, label="N(μ={}, σ={})".format(mu1, sigma1))
    plot(m, q, label="N(μ={}, σ={})".format(mu2, sigma2))
    
    legend(loc="best")
    xlabel("m")
    ylabel("p(m)")
    title("D( N1 || N2 ) = {:.03f}   und   D( N2 || N1 ) = {:.03f}".format(klpq, klqp))
    
    sns.despine()
    show()
    
interactive_plot = widgets.interactive(kkl_example,
    mu1 = widgets.FloatSlider(min=-5, max=5, step=.01, value=-2),
    sigma1 = widgets.FloatSlider(min=0, max=5, step=.01, value=1.5),
    mu2 = widgets.FloatSlider(min=-5, max=5, step=.01, value=2),
    sigma2 = widgets.FloatSlider(min=0, max=5, step=.01, value=1.5))

output = interactive_plot.children[-1]
output.layout.height = '410px'
interactive_plot